In [ ]:
#Crimes_2012-2015
#https://catalog.data.gov/dataset/crimes-2012-2015

# 935259 Rows
# 2012 - 2015
#Header = Date.Rptd, DR.NO,DATE.OCC,TIME.OCC,AREA,AREA.NAME,RD,Crm.Cd,CrmCd.Desc,Status,Status.Desc,
#            LOCATION,Cross.Street,Location.1
#AREA.NAME = districten


years= [2012, 2013, 2014, 2015]

#crime_data = []

#import dataset (pas locatie aan)
#filter header eruit
# crime_data = (sc.textFile('../desktop/Crimes_2012-2015').filter(lambda l: 'DATE.OCC' not in l))

# convert to a dataframe
def convert_to_df(year):
#     if year == '2015-01':
#         separator = '\t'
#     else:
#         separator = ';'
    seperator = ";"
    crime_data = sc.textFile('../desktop/Crimes_2012-2015')
    year_rdd = (crime_data
                    .filter(lambda l: 'DATE.OCC' not in l)
                    .map(lambda l: add_fields(l, separator))
               )
    
    # apply the schema to the RDD.
    month_df = sqlContext.createDataFrame(month_rdd, schema)
    
    return month_df

fields = []
fields.append(StructField('DateReported', DateTime(), True))
fields.append(StructField('Docnr', IntegerType(), True))
fields.append(StructField('DateOccupied', IntegerType(), True))
fields.append(StructField('areaId', IntegerType(), True))
fields.append(StructField('areaName', IntegerType(), True))
fields.append(StructField('rd', IntegerType(), True))
fields.append(StructField('crimeId', IntegerType(), True))
fields.append(StructField('crimeName', StringType(), True))
fields.append(StructField('statusId', IntegerType(), True))
fields.append(StructField('statusName', FloatType(), True))
fields.append(StructField('location', FloatType(), True))
fields.append(StructField('crossStreet', FloatType(), True))
fields.append(StructField('location_type', StringType(), True))
schema = StructType(fields)

for i in range(1, 12):
    try:
        month = '2015-' + str(i).zfill(2)
        convert_to_df(month).registerTempTable(MONTHS[i - 1] + '_table')
        sql = 'SELECT COUNT(*) AS CNT, SUM(location_length) AS SUM FROM ' + MONTHS[i - 1] + '_table WHERE location_type = \'A\''
        bike_data.append(sqlContext.sql(sql).collect())
    except Exception as e:
        bike_data.append(None)

        
#
def add_fields(line, separator):
    fields = []
    vals = line.split(separator)
    # bike id
    fields.append(int(vals[1]))
    # day
    if '.' in vals[9]:
        date = datetime.strptime(vals[9][:-4], '%Y-%m-%d %H:%M:%S')
    else:
        date = datetime.strptime(vals[9], '%Y-%m-%d %H:%M:%S')
    fields.append(date.day)
    # month
    fields.append(date.month)
    # weekend
    if date.weekday() == 5 or date.weekday() == 6:
        fields.append(1)
    else:
        fields.append(0)
    # hour
    fields.append(date.hour)
    # minute
    fields.append(date.minute)
    try:
        # station ID
        fields.append(int(vals[4]))
        # station data
        sd = station_dict.value
        e = []
        d = sd[int(vals[4])]
        e.append(str(d[0]))
        e.append(int(d[1]))
        e.append(float(d[2]))
        e.append(float(d[3]))
        fields.extend(e)
    except:
        fields.extend([-1, 'Unknown', 0, 0.0, 0.0])
    # location length
    try:
        fields.append(float(vals[18]))
    except:
        fields.append(0.0)
    # location type
    fields.append(vals[8])
    
    return fields